# Lecture 16: Imputation and Entity Resolution

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

import importlib
importlib.reload(mpl); importlib.reload(plt); importlib.reload(sns)

sns.reset_orig()
sns.set(font_scale=1.5)
%matplotlib inline
%config SqlMagic.displaylimit = 100

# GNIS data

This is the GNIS dataset from the previous lecture. If you didn't load in the database, run the below cell before connecting.

In [2]:
!psql -h localhost -d gnis -c 'SELECT pg_terminate_backend(pg_stat_activity.pid) FROM pg_stat_activity WHERE datname = current_database() AND pid <> pg_backend_pid();'
!psql -h localhost -c 'DROP DATABASE IF EXISTS gnis'
!psql -h localhost -c 'CREATE DATABASE gnis' 
!psql -h localhost -d gnis -f ../lec14/data/gnis.sql

 pg_terminate_backend 
----------------------
(0 rows)

DROP DATABASE
CREATE DATABASE
SET
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
SET
SET
SET
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
COPY 3195
COPY 11533
CREATE INDEX


In [3]:
%reload_ext sql
%sql postgresql://localhost:5432/gnis

There's a new jupysql version available (0.10.10), you're running 0.10.0. To upgrade: pip install jupysql --upgrade
Deploy Streamlit apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


## Make a "holey" dataset

In [4]:
%sql SELECT setseed(0.12345);

Running query in 'postgresql://localhost:5432/gnis'

1 rows affected.

setseed
""


In [5]:
%%sql
DROP TABLE IF EXISTS holey CASCADE;
CREATE TABLE holey AS 
SELECT feature_id, feature_name, feature_class, state_alpha, county_name, 
       prim_lat_dec, prim_long_dec, 
       CASE WHEN random() > 0.9 THEN NULL
            ELSE elev_in_m
        END AS elev_in_m
  FROM national;
SELECT count(elev_in_m)::float / count(*) FROM holey;

Running query in 'postgresql://localhost:5432/gnis'

11533 rows affected.

1 rows affected.

?column?
0.8755744385675887


In [6]:
%%sql
SELECT * FROM holey TABLESAMPLE BERNOULLI(.1);

Running query in 'postgresql://localhost:5432/gnis'

11 rows affected.

feature_id,feature_name,feature_class,state_alpha,county_name,prim_lat_dec,prim_long_dec,elev_in_m
322312,Saint Stephens Church,Church,GA,Lee,31.6590624,-84.0793506,None
343973,Hopewell Cemetery,Cemetery,GA,Colquitt,31.1069208,-83.7225542,84.0
810480,03S44E03ACA_01 Well,Well,MT,Rosebud,45.6083332,-106.2822366,None
814008,04N02E18ACAC01 Well,Well,MT,Broadwater,46.1021509,-111.5238582,None
976346,Glasby Creek,Stream,NY,St. Lawrence,44.0764516,-74.9201867,485.0
1138211,Black Rock,Lava,OR,Klamath,43.5162352,-121.8083604,1454.0
1517973,Columbia,Populated Place,WA,Whatcom,48.9670667,-122.162928,169.0
1564259,Eagle Rock,Pillar,WI,Vernon,43.4285876,-90.7787408,337.0
1896578,Shakes Village,Locale,AK,Wrangell,55.9666667,-132.2166667,372.0
1995508,Lozier Post Office (historical),Post Office,IA,Woodbury,42.5472121,-95.9002918,405.0


## Schema 1. Default Value Imputation with SQL

In [7]:
%%sql
WITH elevavg AS (SELECT avg(elev_in_m) AS av_elev FROM holey)
SELECT h.*, 
       CASE WHEN h.elev_in_m IS NOT NULL THEN h.elev_in_m::integer
            ELSE e.av_elev::integer
        END AS imputed_elev_in_m
  FROM holey h, elevavg e
LIMIT 10;

Running query in 'postgresql://localhost:5432/gnis'

10 rows affected.

feature_id,feature_name,feature_class,state_alpha,county_name,prim_lat_dec,prim_long_dec,elev_in_m,imputed_elev_in_m
1230,Belmont Mountains,Range,AZ,Maricopa,33.642258,-112.9010129,931.0,931
1839,Bootlegger Saddle,Gap,AZ,Cochise,31.8931474,-109.2831176,2707.0,2707
2336,Cabeza Prieta Game Range,Park,AZ,Yuma,32.250056,-113.45074,275.0,275
2750,Chandler Springs,Spring,AZ,Navajo,35.3766788,-110.4754096,1685.0,1685
3032,Cline Well,Well,AZ,Cochise,31.9000849,-110.3428525,None,484
3039,Clover Tank,Reservoir,AZ,Gila,33.8509816,-110.2577249,1563.0,1563
3060,Coat Spring,Spring,AZ,Navajo,36.12678,-110.3330424,1926.0,1926
3143,Comar Spring,Spring,AZ,Navajo,35.5308428,-110.4162424,1732.0,1732
3333,Cottonwood Creek,Stream,AZ,Coconino,36.050817,-111.9865535,800.0,800
3342,Cottonwood Creek,Stream,AZ,Mohave,36.901931,-112.5632547,1389.0,1389


## Scheme 2. Correlation across columns

In [8]:
# Training is an aggregate function
# Here we'll train the model in SQL just for fun
result = %sql SELECT regr_slope(elev_in_m, prim_long_dec), \
               regr_intercept(elev_in_m, prim_long_dec) FROM holey
slope, intercept = result[0]
slope, intercept

Running query in 'postgresql://localhost:5432/gnis'

1 rows affected.

(-10.314179001097786, -477.9603219322606)

In [9]:
%%sql
SELECT prim_long_dec, elev_in_m
FROM holey
WHERE prim_long_dec >= -122
ORDER BY prim_long_dec
OFFSET 40;

Running query in 'postgresql://localhost:5432/gnis'

10834 rows affected.

prim_long_dec,elev_in_m
-121.8142546,469.0
-121.8099602,None
-121.8083604,1454.0
-121.8056805,None
-121.803556,1614.0
-121.7974945,1264.0
-121.7960906,219.0
-121.784236,1684.0
-121.7832846,2.0
-121.7785678,46.0


In [10]:
%%sql
SELECT *, 
       CASE WHEN elev_in_m IS NOT NULL THEN elev_in_m::integer
            ELSE ({{slope}}*prim_long_dec + {{intercept}})::integer
        END AS imputed_elev_in_m
  FROM holey
LIMIT 10;

Running query in 'postgresql://localhost:5432/gnis'

10 rows affected.

feature_id,feature_name,feature_class,state_alpha,county_name,prim_lat_dec,prim_long_dec,elev_in_m,imputed_elev_in_m
1230,Belmont Mountains,Range,AZ,Maricopa,33.642258,-112.9010129,931.0,931
1839,Bootlegger Saddle,Gap,AZ,Cochise,31.8931474,-109.2831176,2707.0,2707
2336,Cabeza Prieta Game Range,Park,AZ,Yuma,32.250056,-113.45074,275.0,275
2750,Chandler Springs,Spring,AZ,Navajo,35.3766788,-110.4754096,1685.0,1685
3032,Cline Well,Well,AZ,Cochise,31.9000849,-110.3428525,None,660
3039,Clover Tank,Reservoir,AZ,Gila,33.8509816,-110.2577249,1563.0,1563
3060,Coat Spring,Spring,AZ,Navajo,36.12678,-110.3330424,1926.0,1926
3143,Comar Spring,Spring,AZ,Navajo,35.5308428,-110.4162424,1732.0,1732
3333,Cottonwood Creek,Stream,AZ,Coconino,36.050817,-111.9865535,800.0,800
3342,Cottonwood Creek,Stream,AZ,Mohave,36.901931,-112.5632547,1389.0,1389


## Scheme 3. General model-based interpolation

We won't show the demo; check slides for the general idea.

## Scheme 4. [simple] Correlation across ordered rows

In [11]:
%%sql
-- The following does not work in PostgreSQL!
WITH buggy AS (
SELECT *,
       CASE WHEN elev_in_m IS NOT NULL THEN elev_in_m
            ELSE lag(elev_in_m, 1)
                 OVER (ORDER BY feature_id)
        END AS imputed_elev_in_m
  FROM holey
)
SELECT feature_id, elev_in_m, imputed_elev_in_m FROM buggy ORDER BY feature_id ASC LIMIT 12 OFFSET 183;

Running query in 'postgresql://localhost:5432/gnis'

12 rows affected.

feature_id,elev_in_m,imputed_elev_in_m
48638,46.0,46.0
49192,401.0,401.0
49214,194.0,194.0
49350,None,194.0
49578,None,None
49802,47.0,47.0
49925,111.0,111.0
50059,71.0,71.0
50309,None,71.0
50661,26.0,26.0


In [12]:
%%sql
-- Here's a UDA fix from
-- https://stackoverflow.com/questions/18987791/how-do-i-efficiently-select-the-previous-non-null-value
CREATE OR REPLACE FUNCTION coalesce_agg_sfunc(state anyelement, value anyelement) RETURNS anyelement AS
$$
    SELECT coalesce(value, state);
$$ LANGUAGE SQL;

CREATE OR REPLACE AGGREGATE coalesce_agg(anyelement) (
    SFUNC = coalesce_agg_sfunc,
    STYPE  = anyelement);

Running query in 'postgresql://localhost:5432/gnis'

++
||
++
++

In [13]:
%%sql
-- Fixed to handle repeated NULLs
WITH fixed AS (
SELECT *,
       coalesce_agg(elev_in_m) OVER (order by feature_id) AS imputed_elev_in_m
  FROM holey
)
SELECT feature_id, elev_in_m, imputed_elev_in_m FROM fixed ORDER BY feature_id ASC LIMIT 12 OFFSET 183;

Running query in 'postgresql://localhost:5432/gnis'

12 rows affected.

feature_id,elev_in_m,imputed_elev_in_m
48638,46.0,46.0
49192,401.0,401.0
49214,194.0,194.0
49350,None,194.0
49578,None,194.0
49802,47.0,47.0
49925,111.0,111.0
50059,71.0,71.0
50309,None,71.0
50661,26.0,26.0


## Generalized interpolation across rows

Forward pass

In [14]:
%%sql
-- 1. Forward assign run numbers to rows, propagate val, get nextval
CREATE OR REPLACE VIEW forward AS
SELECT feature_id, elev_in_m,
       SUM(CASE WHEN elev_in_m IS NULL THEN 0 ELSE 1 END) 
         OVER (ORDER BY feature_id) AS run,
       coalesce_agg(elev_in_m) OVER (ORDER BY feature_id) AS run_start,
       CASE WHEN elev_in_m IS NULL 
              THEN lead(elev_in_m, 1) OVER (ORDER BY feature_id)
            ELSE NULL
             END AS nextval
  FROM holey;
SELECT * FROM forward ORDER BY feature_id ASC LIMIT 12 OFFSET 183;

Running query in 'postgresql://localhost:5432/gnis'

12 rows affected.

feature_id,elev_in_m,run,run_start,nextval
48638,46.0,168,46.0,None
49192,401.0,169,401.0,None
49214,194.0,170,194.0,None
49350,None,170,194.0,None
49578,None,170,194.0,47.0
49802,47.0,171,47.0,None
49925,111.0,172,111.0,None
50059,71.0,173,71.0,None
50309,None,173,71.0,26.0
50661,26.0,174,26.0,None


Backward pass

In [15]:
%%sql
-- 2. Backward: assign run_end, run_size, run_rank
CREATE OR REPLACE VIEW backward AS
SELECT *,
       CASE WHEN elev_in_m IS NOT NULL THEN elev_in_m
            ELSE coalesce_agg(nextval) OVER (PARTITION BY run ORDER BY feature_id DESC)
        END AS run_end,
       count(*) OVER (PARTITION BY run) AS run_size,
       -1 + (RANK() OVER (PARTITION BY run ORDER BY feature_id)) AS run_rank
  FROM forward;
SELECT * FROM backward ORDER BY feature_id ASC LIMIT 12 OFFSET 183;

Running query in 'postgresql://localhost:5432/gnis'

12 rows affected.

feature_id,elev_in_m,run,run_start,nextval,run_end,run_size,run_rank
48638,46.0,168,46.0,None,46.0,1,0
49192,401.0,169,401.0,None,401.0,1,0
49214,194.0,170,194.0,None,194.0,3,0
49350,None,170,194.0,None,47.0,3,1
49578,None,170,194.0,47.0,47.0,3,2
49802,47.0,171,47.0,None,47.0,1,0
49925,111.0,172,111.0,None,111.0,1,0
50059,71.0,173,71.0,None,71.0,2,0
50309,None,173,71.0,26.0,26.0,2,1
50661,26.0,174,26.0,None,26.0,1,0


Scalar function pass

In [16]:
%%sql
-- 3. Simple scalar pass
CREATE OR REPLACE VIEW final AS
SELECT *, 
       run_start + run_rank*((run_end-run_start)/(run_size))
         AS interpolated
  FROM backward;
SELECT * FROM final  ORDER BY feature_id ASC LIMIT 12 OFFSET 183;

Running query in 'postgresql://localhost:5432/gnis'

12 rows affected.

feature_id,elev_in_m,run,run_start,nextval,run_end,run_size,run_rank,interpolated
48638,46.0,168,46.0,None,46.0,1,0,46.0
49192,401.0,169,401.0,None,401.0,1,0,401.0
49214,194.0,170,194.0,None,194.0,3,0,194.0
49350,None,170,194.0,None,47.0,3,1,145.0
49578,None,170,194.0,47.0,47.0,3,2,96.0
49802,47.0,171,47.0,None,47.0,1,0,47.0
49925,111.0,172,111.0,None,111.0,1,0,111.0
50059,71.0,173,71.0,None,71.0,2,0,71.0
50309,None,173,71.0,26.0,26.0,2,1,48.5
50661,26.0,174,26.0,None,26.0,1,0,26.0


In [17]:
%sql EXPLAIN Analyze SELECT * from final LIMIT 500;

Running query in 'postgresql://localhost:5432/gnis'

20 rows affected.

QUERY PLAN
Limit (cost=2196.29..2267.30 rows=500 width=72) (actual time=28.726..30.276 rows=500 loops=1)
-> Subquery Scan on backward (cost=2196.29..3834.15 rows=11533 width=72) (actual time=28.724..30.244 rows=500 loops=1)
-> WindowAgg (cost=2196.29..3545.82 rows=11533 width=64) (actual time=28.722..30.179 rows=500 loops=1)
-> WindowAgg (cost=2196.29..3344.00 rows=11533 width=56) (actual time=28.716..29.878 rows=501 loops=1)
-> Incremental Sort (cost=2196.29..3142.17 rows=11533 width=48) (actual time=28.713..29.627 rows=502 loops=1)
"Sort Key: forward.run, forward.feature_id"
Presorted Key: forward.run
Full-sort Groups: 16 Sort Method: quicksort Average Memory: 27kB Peak Memory: 27kB
-> WindowAgg (cost=2192.36..2423.02 rows=11533 width=48) (actual time=28.644..29.500 rows=514 loops=1)
-> Sort (cost=2192.36..2221.19 rows=11533 width=40) (actual time=28.580..28.615 rows=515 loops=1)


# String distance

In [18]:
%reload_ext sql
%sql postgresql://localhost:5432/postgres

Connecting and switching to connection postgresql://localhost:5432/postgres

In [19]:
%sql CREATE EXTENSION IF NOT EXISTS fuzzystrmatch;

Running query in 'postgresql://localhost:5432/postgres'

++
||
++
++

In [20]:
%%sql
DROP TABLE IF EXISTS Strings;
CREATE TABLE Strings (str1 TEXT, str2 TEXT);
INSERT INTO Strings VALUES
 ('Lisa', 'List'),
 ('Lisa', 'License'),
 ('Joe', 'Noel');

Running query in 'postgresql://localhost:5432/postgres'

3 rows affected.

++
||
++
++

In [21]:
%%sql
SELECT *,
       levenshtein(str1, str2),
       soundex(str1) as soundex1,
       soundex(str2) as soundex2,
       dmetaphone(str1) AS dmetaphone1,
       dmetaphone(str2) AS dmetaphone2,
       dmetaphone_alt(str1) AS dmetaphone_alt1,
       dmetaphone_alt(str2) AS dmetaphone_alt2
FROM Strings;

Running query in 'postgresql://localhost:5432/postgres'

3 rows affected.

str1,str2,levenshtein,soundex1,soundex2,dmetaphone1,dmetaphone2,dmetaphone_alt1,dmetaphone_alt2
Lisa,List,1,L200,L230,LS,LST,LS,LST
Lisa,License,4,L200,L252,LS,LSNS,LS,LSNS
Joe,Noel,2,J000,N400,J,NL,A,NL
